In [31]:
import pywt
import os
import pandas as pd
import librosa
import librosa.display
import glob 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from fitter import Fitter, get_common_distributions, get_distributions
from matplotlib.ticker import FormatStrFormatter
from pydub import AudioSegment, effects  
import pickle
import collections
import re

from sklearn.preprocessing import StandardScaler
import sklearn

from utils import plot_projections

from scipy.stats import kurtosis, skew

%matplotlib widget

In [24]:
def extract_speaker_num(f):
    
    substr = f.split('/')[7].split('_')
    
    speaker_str = re.split('(\d+)', substr[0])
    
#     print(substr[1].split('.')[0])
    
#     print(speaker_str + substr[1].split('.')[0])
    
    return int(speaker_str[1] + str(substr[1].split('.')[0]))

In [25]:
extract_speaker_num('/home/hashim/PHD/audio_data/AllAudioSamples/Laser_Male_Female/speaker10_001.wav')

10001

In [26]:
laser_dir = '/home/hashim/PHD/audio_data/AllAudioSamples/Laser_Male_Female/'
laser_files = list(glob.glob(os.path.join(laser_dir, '*.wav')))
laser_files.sort(key=extract_speaker_num)
print(laser_files)
print(len(laser_files))

['/home/hashim/PHD/audio_data/AllAudioSamples/Laser_Male_Female/speaker1_001.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Laser_Male_Female/speaker1_002.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Laser_Male_Female/speaker1_003.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Laser_Male_Female/speaker1_004.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Laser_Male_Female/speaker1_005.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Laser_Male_Female/speaker2_001.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Laser_Male_Female/speaker2_002.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Laser_Male_Female/speaker2_003.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Laser_Male_Female/speaker2_004.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Laser_Male_Female/speaker2_005.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Laser_Male_Female/speaker3_001.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Laser_Male_Female/speaker3_002.wav', '/home/hashim/P

In [27]:
orig_dir = '/home/hashim/PHD/audio_data/AllAudioSamples/Original_Male_Female/'
orig_files = list(glob.glob(os.path.join(orig_dir, '*wav')))
orig_files.sort(key=extract_speaker_num)
print(orig_files)
print(len(orig_files))

['/home/hashim/PHD/audio_data/AllAudioSamples/Original_Male_Female/speaker1_001.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Original_Male_Female/speaker1_002.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Original_Male_Female/speaker1_003.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Original_Male_Female/speaker1_004.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Original_Male_Female/speaker1_005.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Original_Male_Female/speaker2_001.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Original_Male_Female/speaker2_002.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Original_Male_Female/speaker2_003.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Original_Male_Female/speaker2_004.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Original_Male_Female/speaker2_005.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Original_Male_Female/speaker3_001.wav', '/home/hashim/PHD/audio_data/AllAudioSamples/Original_Male_Femal

In [28]:
sample_id = 10
laser_sample = laser_files[sample_id]
gen_sample = orig_files[sample_id]

In [29]:
audio_data, sr = librosa.load(laser_sample, res_type='kaiser_fast')

In [30]:
print(len(audio_data))
audio_data

100725


array([ 0.00026551,  0.00042113,  0.00012006, ..., -0.00093125,
       -0.00022951,  0.        ], dtype=float32)

In [32]:
# stitching the laser and original audios and saving them

out_folder = 'combined_male_female_speaker/'

out_dir = '/home/hashim/PHD/audio_data/AllAudioSamples/' + out_folder

if not os.path.exists(out_dir):
    
    os.makedirs(out_dir)
    print("The new directory is created!")
    
for i in range(len(laser_files)):
    
    original_audio = effects.normalize(AudioSegment.from_file(orig_files[i], format="wav"))
    laser_audio = effects.normalize(AudioSegment.from_file(laser_files[i], format="wav"))

    combined_audio = original_audio[:2000] + laser_audio[2000:]
    
    out_file = out_dir + 'original_laser_' + os.path.basename(os.path.normpath(orig_files[i]))
    
    print(out_file)

    file_handle = combined_audio.export(out_file, format="wav")

/home/hashim/PHD/audio_data/AllAudioSamples/combined_male_female_speaker/original_laser_speaker1_001.wav
/home/hashim/PHD/audio_data/AllAudioSamples/combined_male_female_speaker/original_laser_speaker1_002.wav
/home/hashim/PHD/audio_data/AllAudioSamples/combined_male_female_speaker/original_laser_speaker1_003.wav
/home/hashim/PHD/audio_data/AllAudioSamples/combined_male_female_speaker/original_laser_speaker1_004.wav
/home/hashim/PHD/audio_data/AllAudioSamples/combined_male_female_speaker/original_laser_speaker1_005.wav
/home/hashim/PHD/audio_data/AllAudioSamples/combined_male_female_speaker/original_laser_speaker2_001.wav
/home/hashim/PHD/audio_data/AllAudioSamples/combined_male_female_speaker/original_laser_speaker2_002.wav
/home/hashim/PHD/audio_data/AllAudioSamples/combined_male_female_speaker/original_laser_speaker2_003.wav
/home/hashim/PHD/audio_data/AllAudioSamples/combined_male_female_speaker/original_laser_speaker2_004.wav
/home/hashim/PHD/audio_data/AllAudioSamples/combined_ma

In [ ]:
model_dir = 'models/'
model_filename = 'svm_model.sav'

In [ ]:
# loading combined file paths

comb_dir = '/home/hashim/PHD/audio_data/AllAudioSamples/combined/'
comb_files = list(glob.glob(os.path.join(comb_dir, '*wav')))
comb_files.sort()
print(comb_files)
print(len(comb_files))

In [ ]:
db1 = pywt.Wavelet('db1')

## Computing Features From the Combined Audios

In [ ]:
%%capture
# compute coefficients for both laser and then compute variance, skewness and kurtosis

comb_sigma_feat = []
comb_skew_feat = []
comb_kurt_feat = []

for combf in comb_files:
    audio_data, sr = librosa.load(combf, res_type='kaiser_fast')
#     stft = librosa.stft(audio_data, n_fft=256)
#     S_db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
    coeffs = pywt.wavedec(audio_data, db1, mode='constant', level=5)
    
    
    sigma_feat = []
    skew_feat = []
    kurt_feat = []
    
    for cf in coeffs:
        
#         cf = cf - np.mean(cf)
        
        sigma_feat.append(np.var(cf, ddof=1))
        skew_feat.append(skew(cf))
        kurt_feat.append(kurtosis(cf))
        
    comb_sigma_feat.append(sigma_feat)
    comb_skew_feat.append(skew_feat)
    comb_kurt_feat.append(kurt_feat)    
    
comb_sigma_feat = np.array(comb_sigma_feat)
comb_skew_feat = np.array(comb_skew_feat)
comb_kurt_feat = np.array(comb_kurt_feat)

In [ ]:
print(comb_sigma_feat.shape)
print(comb_skew_feat.shape)
print(comb_kurt_feat.shape)

## Prediction Using already trained model

In [ ]:
import sklearn

from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import plot_confusion_matrix

In [ ]:
X_comb = np.concatenate((comb_sigma_feat, comb_sigma_feat, comb_kurt_feat), axis=1)
comb_label = np.array(['laser']*X_comb.shape[0])
print(comb_label)
print(comb_label.shape)
X_comb.shape

In [ ]:
scaler = StandardScaler().fit(X_comb)
X_comb = scaler.transform(X_comb)

In [ ]:
# loading the saved model
model_svm = pickle.load(open(model_dir + model_filename, 'rb'))

In [ ]:
y_pred_comb = model_svm.predict(X_comb)

In [ ]:
y_pred_comb

In [ ]:
cm = confusion_matrix(comb_label, y_pred_comb)

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model_svm.classes_)
disp.plot()

# plt.savefig("ConfusionMatrix", dpi=300)

In [ ]:
model_svm.classes_

In [ ]:
print("accuracy = ", accuracy_score(comb_label, y_pred_comb))

## Frame by Frame Analysis

In [ ]:
frame_len = 1

In [ ]:
def audiosegment_to_librosawav(audiosegment):    
    channel_sounds = audiosegment.split_to_mono()
    samples = [s.get_array_of_samples() for s in channel_sounds]

    fp_arr = np.array(samples).T.astype(np.float32)
    fp_arr /= np.iinfo(samples[0].typecode).max
    fp_arr = fp_arr.reshape(-1)

    return fp_arr

In [ ]:
comb_true_labels = []
    
# comb_sigma_feat = np.array([])
# comb_skew_feat = np.array([])
# comb_kurt_feat = np.array([])

combf_dict = collections.defaultdict(dict)


for i, combf in enumerate(comb_files):
    
    comb_audio = AudioSegment.from_file(combf, format="wav")
    
    # split comb audio in 1-second slices
    comb_slices_1sec = comb_audio[::1000]
    
#     samples = comb_audio.get_array_of_samples()

    true_labels = []
    
    sigma_feat_slice = []
    skew_feat_slice = []
    kurt_feat_slice = []

    for j, comb_slice in enumerate(comb_slices_1sec):
        
        # converting to numpy float32 array
#         channel_sounds = comb_slice.split_to_mono()
#         samples = [s.get_array_of_samples() for s in channel_sounds]
        
#         fp_arr = np.array(samples).T.astype(np.float32)
#         fp_arr /= np.iinfo(samples[0].typecode).max
        
        fp_arr = audiosegment_to_librosawav(comb_slice)
        
        coeffs = pywt.wavedec(fp_arr, db1, mode='constant', level=5)
        
        sigma_feat = []
        skew_feat = []
        kurt_feat = []

        for cf in coeffs:

            sigma_feat.append(np.var(cf, ddof=1))
            skew_feat.append(skew(cf))
            kurt_feat.append(kurtosis(cf))

        sigma_feat_slice.append(sigma_feat)
        skew_feat_slice.append(skew_feat)
        kurt_feat_slice.append(kurt_feat) 

#         comb_sigma_feat[i][j] = sigma_feat
#         comb_skew_feat[i][j] = skew_feat
#         comb_kurt_feat[i][j] = kurt_feat
        
        if j < 2:
            true_labels.append('original')
        else:
            true_labels.append('laser')
            
#         if j == 1:
#             true_labels.append('laser')
#         else:
#             true_labels.append('original')
                
#         i+=1
#         print(j)
    
#     comb_true_labels.append(true_labels)
#     comb_sigma_feat.append(sigma_feat_slice)
#     comb_skew_feat.append(skew_feat_slice)
#     comb_kurt_feat.append(kurt_feat_slice)

    
    sigma_feat_slice = np.squeeze(np.array(sigma_feat_slice))
    skew_feat_slice = np.squeeze(np.array(skew_feat_slice))
    kurt_feat_slice = np.squeeze(np.array(kurt_feat_slice))
    
    print(sigma_feat_slice.shape)
    print(skew_feat_slice.shape)
    print(kurt_feat_slice.shape)

    combf_dict[i]['feat'] = np.concatenate((sigma_feat_slice, skew_feat_slice, kurt_feat_slice), axis=1)
    combf_dict[i]['labels'] = true_labels
    
# comb_sigma_feat = np.array(comb_sigma_feat)
# comb_skew_feat = np.array(comb_skew_feat)
# comb_kurt_feat = np.array(comb_kurt_feat)

In [ ]:
combf_dict

In [ ]:
# accuracy rate (frame error rate)

# frame size using is multiple of the actual original audio

# typical utterance is 10 ms for humans (multiples of that)

# too samll window gives better time resolution but will be more computationally expensive but

# framze size increases, computatilaly better but lower accuracy, border frames are bound to be uncertain untill unless your analysis window is multiple of the injected audio (by stroke of luck)

# use overlapping windows

## Predicting the labels on each file

In [ ]:
combf_keys = combf_dict.keys()
combf_keys

In [ ]:
# fig, axes = plt.subplots(nrows=25, ncols=2, figsize=(8, 80))

pred_label_dict = dict()

accuracy_dict = dict()

for key in combf_dict:
    
    X_comb = combf_dict[key]['feat']
    comb_labels = combf_dict[key]['labels']
    
    scaler = StandardScaler().fit(X_comb)
    X_comb = scaler.transform(X_comb)
    
    pred_labels = model_svm.predict(X_comb)
    
    accuracy_dict[key] = accuracy_score(comb_labels, pred_labels)
    
#     print("True Labels = ", comb_labels)
#     print("Predicted Labels = ", pred_labels)

#     plot_confusion_matrix(model_svm, X_comb, comb_labels, ax=ax, display_labels=['laser', 'original'])

#     cm = confusion_matrix(comb_labels, pred_labels)
#     disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model_svm.classes_)
#     disp.plot(ax=ax)

# plt.tight_layout()  
# plt.show()

In [ ]:
accuracy_dict

avg_accuracy = sum(accuracy_dict.values())/len(accuracy_dict)
avg_accuracy

In [ ]:
accuracy_dict

In [ ]:
# dig deep into a single frame 
# if a certain frame is totally original and labelled as laser and vice versa

# why those frames are predicted wrong

# Highlight the border frames

# if we take out the border frames, what will be accuracy then

# probablisitic decision instead of a binary decision (future)

# Do 50% sliding

# Theoretical in the paper(tw -> advance window, ti -> injection window)

# motivation behind 1 sec window is that our experimental investigation says that 'Hey Google' is a 1 sec utterance for most of the speakers.

## Frame by Frame Analysis Using LogVar 

In [ ]:
comb_true_labels = []
    
# comb_sigma_feat = np.array([])
# comb_skew_feat = np.array([])
# comb_kurt_feat = np.array([])

combf_dict = collections.defaultdict(dict)


for i, combf in enumerate(comb_files):
    
    comb_audio = AudioSegment.from_file(combf, format="wav")
    
    # split comb audio in 1-second slices
    comb_slices_1sec = comb_audio[::1000]
    
#     samples = comb_audio.get_array_of_samples()

    true_labels = []
    
    sigma_feat_slice = []
    skew_feat_slice = []
    kurt_feat_slice = []

    for j, comb_slice in enumerate(comb_slices_1sec):
        
        # converting to numpy float32 array
#         channel_sounds = comb_slice.split_to_mono()
#         samples = [s.get_array_of_samples() for s in channel_sounds]
        
#         fp_arr = np.array(samples).T.astype(np.float32)
#         fp_arr /= np.iinfo(samples[0].typecode).max
        
        fp_arr = audiosegment_to_librosawav(comb_slice)
        
        coeffs = pywt.wavedec(fp_arr, db1, mode='constant', level=5)
        
        sigma_feat = []
        skew_feat = []
        kurt_feat = []

        for cf in coeffs:

            sigma_feat.append(np.log(np.var(cf)))
            skew_feat.append(skew(cf))
            kurt_feat.append(kurtosis(cf))

        sigma_feat_slice.append(sigma_feat)
        skew_feat_slice.append(skew_feat)
        kurt_feat_slice.append(kurt_feat) 

#         comb_sigma_feat[i][j] = sigma_feat
#         comb_skew_feat[i][j] = skew_feat
#         comb_kurt_feat[i][j] = kurt_feat
        
        if j < 2:
            true_labels.append('original')
        else:
            true_labels.append('laser')
            
#         if j == 1:
#             true_labels.append('laser')
#         else:
#             true_labels.append('original')
                
#         i+=1
#         print(j)
    
#     comb_true_labels.append(true_labels)
#     comb_sigma_feat.append(sigma_feat_slice)
#     comb_skew_feat.append(skew_feat_slice)
#     comb_kurt_feat.append(kurt_feat_slice)

    
    sigma_feat_slice = np.squeeze(np.array(sigma_feat_slice))
    skew_feat_slice = np.squeeze(np.array(skew_feat_slice))
    kurt_feat_slice = np.squeeze(np.array(kurt_feat_slice))
    
#     print(sigma_feat_slice.shape)
#     print(skew_feat_slice.shape)
#     print(kurt_feat_slice.shape)

    combf_dict[i]['feat'] = np.concatenate((sigma_feat_slice, skew_feat_slice, kurt_feat_slice), axis=1)
    combf_dict[i]['labels'] = true_labels
    
# comb_sigma_feat = np.array(comb_sigma_feat)
# comb_skew_feat = np.array(comb_skew_feat)
# comb_kurt_feat = np.array(comb_kurt_feat)

## Predicting the labels on each file for Log Var data

In [ ]:
model_dir = 'models/'
model_filename = 'svm_model_log_sigma.sav'

In [ ]:
# loading the saved log var model
model_svm_logvar = pickle.load(open(model_dir + model_filename, 'rb'))

In [ ]:
fig, axes = plt.subplots(nrows=25, ncols=2, figsize=(8, 80))

pred_label_dict = dict()

accuracy_dict = dict()

for key, ax in zip(combf_dict, axes.flatten()):
    
    X_comb = combf_dict[key]['feat']
    comb_labels = combf_dict[key]['labels']
    
#     scaler = StandardScaler().fit(X_comb)
#     X_comb = scaler.transform(X_comb)
    
    pred_labels = model_svm_logvar.predict(X_comb)
    
    accuracy_dict[key] = accuracy_score(comb_labels, pred_labels)

    cm = confusion_matrix(comb_labels, pred_labels)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model_svm.classes_)
    disp.plot(ax=ax)

In [ ]:
accuracy_dict

avg_accuracy = sum(accuracy_dict.values())/len(accuracy_dict)
avg_accuracy